# Federated Learning Problem Set

Through the course of this problem set, you will ........ [fill in later]

First, we will import a different dataset from the one we used in our experiments. For simplicity, let's use fashion_mnist since the preprocessing to be done is the same.

In [ ]:
# Import and preprocess fashion mnist here

Next, write a function which implements the LASG-WK2 update condition as detailed in equation 10 of the LASG paper [find cleaner way to refer to the paper]. Follow the prototype detailed below.

In [ ]:
def lasg_wk2_update(current_weight_grad, last_upload_weight_grad, weights_diff_history, M, c):
    """
    Inputs:
        current_weight_grad - ...
        last_upload_weight_grad - ...
        weights_diff_history - ...
        M - ...
        c - ...
    
    Outputs:
        Either true or false depending whether the update condition was met.
        If true, this means that the worker checking this condition should not upload (and we have saved on communication! yay!)
        If false, this means that the worker should upload its model.

    """

Nice job! You did it, and now you have hopefully gained an understanding of what should be considered when deciding whether a round of communication is worthwhile, as well as how some of the hyperparameters (number of workers and weight vector C) affect the LASG algorithm. 

Now for something a little more interesting. In practice, we observe that Federated Averaging beats out LASG in most cases. Perform a hyperparameter search to find a case (or multiple cases) wherein LASG beats out Federated Averaging (e.g. reaches the same accuracy with fewer rounds of communication). Comment on the case(s) where this occurs - where might this occur in the real world? Why is Federated Learning useful in such a scenario?

In [1]:
# set up actually running fed_avg and LASG with hyperparameter specification available for LASG
# we shouldn't allow variation of fed_avg hyperparameters so that it remains a constant baseline.
# we should include descriptions of what each hyperparameter is so that they know what they are actually varying.

(For CSCI-6961 Students Only) - Implement the LASG-WK1 Algorithm, detailed in Table 2, Algorithm 1 of the LASG paper. This will require using a different update condition (equation (8) in the paper). You may reference the code we wrote for LASG-WK2 in the lasg_wk2() function in the lasg.py file in the github repository [insert link to that file here]. You will need to make some slight changes, since the algorithms are different.

Then, compare the performance of LASG-WK1 and LASG-WK2 on the same dataset with the same hyperparameters. Based on your results, why might you prefer one algorithm over the over?